In [1]:
import json

doc_dir = "data/combinedSemanticDataSet.json"

json_files = json.loads(open("data/combinedSemanticDataSet.json").read())


In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import PreProcessor

document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat", 
    return_embedding=True, 
    duplicate_documents='overwrite'
)

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=50,
    split_respect_sentence_boundary=True,
)

In [2]:
from haystack.nodes import DensePassageRetriever
from haystack.nodes import RAGenerator

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
    )

generator = RAGenerator(
    model_name_or_path="facebook/rag-sequence-nq",
    retriever=retriever,
    top_k=1,
    min_length=2
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagTokenForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagTokenForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagTokenForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-sequence-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [ ]:
from haystack import Pipeline

p = Pipeline()
p.add_node(component=retriever, name="retriever", inputs=['PreProcessor'])
p.add_node(component=generator, name="RAGenerator", inputs=['retriever'])

In [ ]:
p.run(json_files)

In [ ]:
document_store.write_documents(gold)
document_store.update_embeddings(retriever)

In [ ]:
pairs = []
for x in range((int(len(docs) / 2)) - 1):

    pairs.append([docs[(2 * x)].content, docs[(2 * x) + 1].content])

print(pairs[0])

In [ ]:
import dataset

stsb = dataset.map(pairs, 'stsb', split='train')
stsb_dev = datasets.load_dataset(pairs, 'stsb', split='validation')
stsb


In [ ]:
# Saves the document store
document_store.save("my_faiss_index.faiss")

In [ ]:
document_store = FAISSDocumentStore.load("my_faiss_index.faiss")
assert document_store.faiss_index_factory_str == "Flat"